In [112]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [113]:
os.chdir('C:/Users/Administrator/Desktop/ICC machine learning')
data = pd.read_csv('Final project.csv', index_col=0 , error_bad_lines = False)

In [93]:
#data = data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y


In [114]:
df = pd.DataFrame(data)
data.shape
print(data.Loan_Status)
data= pd.get_dummies(data, columns=['Gender'],dummy_na=True)
data= pd.get_dummies(data, columns=['Married'],dummy_na=True)
data= pd.get_dummies(data, columns=['Education'],dummy_na=True)
data= pd.get_dummies(data, columns=['Self_Employed'],dummy_na=True)
data= pd.get_dummies(data, columns=['Property_Area'],dummy_na=True)
data.Loan_Status=data.Loan_Status.replace("Y",1)
data.Loan_Status=data.Loan_Status.replace("N",0)
data.Dependents = data.Dependents.fillna(0)
data.LoanAmount = data.LoanAmount.fillna(0)
data.Loan_Amount_Term = data.Loan_Amount_Term.fillna(0)
data.Credit_History = data.Credit_History.fillna(0)
data.Dependents=data.Dependents.replace("3+",3)

Loan_ID
LP001002    Y
LP001003    N
LP001005    Y
LP001006    Y
LP001008    Y
           ..
LP002978    Y
LP002979    Y
LP002983    Y
LP002984    Y
LP002990    N
Name: Loan_Status, Length: 614, dtype: object


In [115]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [129]:
X_train, X_test, y_train, y_test = train_test_split(data.drop("Loan_Status",axis = 1), data.Loan_Status,
                                                    stratify=data.Loan_Status,
                                                    random_state=0,train_size = 0.75)
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

y_pred=log_reg.predict(X_test)


In [130]:
mislabel = np.sum((y_test!=y_pred))
print("Total number of mislabelled data points from {} test samples is {}".format(len(y_test),mislabel))
from sklearn.metrics import accuracy_score
print("Accuracy of logistic regession model is =",accuracy_score(y_test, y_pred)*100)

Total number of mislabelled data points from 154 test samples is 41
Accuracy of logistic regession model is = 73.37662337662337


In [131]:
from sklearn.metrics import confusion_matrix
loan_status=["With Loan","No Loan"]
cm = (confusion_matrix(y_test,y_pred))
cmdf = pd.DataFrame(cm,index=loan_status,
                    columns=loan_status)
cmdf

,With Loan,No Loan
With Loan,18,30
No Loan,11,95


In [132]:
from sklearn.metrics import precision_score
print("Precision score: {}".format(precision_score(y_test,y_pred)))
from sklearn.metrics import recall_score
print("Recall score: {}".format(recall_score(y_test,y_pred)))

Precision score: 0.76
Recall score: 0.8962264150943396


In [133]:
from sklearn.metrics import f1_score
print("F1 Score: {}".format(f1_score(y_test,y_pred)))

F1 Score: 0.8225108225108226


In [134]:
from sklearn.metrics import classification_report

print("The classification report is as follows...\n")
print(classification_report(y_test, y_pred, target_names=loan_status))

The classification report is as follows...

              precision    recall  f1-score   support

   With Loan       0.62      0.38      0.47        48
     No Loan       0.76      0.90      0.82       106

    accuracy                           0.73       154
   macro avg       0.69      0.64      0.65       154
weighted avg       0.72      0.73      0.71       154



In [135]:
from sklearn.metrics import roc_curve, roc_auc_score
target_probabilities = log_reg.predict_proba(X_test)[:,1]
roc_auc_score(y_test, target_probabilities)

0.8062106918238995